<a href="https://colab.research.google.com/github/jx-dohwan/poc/blob/main/MiniAiffelthon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
#from konlpy.tag import Mecab
from collections import Counter
import re
import os
import random
import math
import urllib.request
import gensim
from gensim.models.keyedvectors import Word2VecKeyedVectors
from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

# mecab = Mecab()

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


https://wikidocs.net/39

In [27]:
import glob
import json
import os

def search(dirname, tv):

    train_summary = []
    train_passage = []
    folder_list = []
    filenames = os.listdir(dirname)
    for filename in filenames:
        full_filename = os.path.join(dirname, filename)
        #print(full_filename)
        if full_filename != dirname+ tv :
          train_filenames = os.listdir(full_filename)
          for train_filename in train_filenames:
            train_full_filename = os.path.join(full_filename, train_filename)
            # print(train_full_filename)
            if train_full_filename == full_filename + '/20per':
              #print(train_full_filename)
              folder_list.append(train_full_filename+'/*.json')
              #print(folder_list)
              #print(folder_list)
    for folder in folder_list: # tqdm추가하기
      for i in glob.glob(folder):
        f = open(i,'r')        
        js = json.loads(f.read())
        train_summary.append(js['Annotation']['summary1'])
        train_passage.append(js['Meta(Refine)']['passage'])
        f.close()
    return train_summary, train_passage


In [ ]:
train_summary, train_passage = search("/content/drive/MyDrive/인공지능/아이펠톤/PoC/data/train", '/TL1.zip')

In [8]:
train_summary[:5]

['자신을 배반한 제 계집과 세상이 엎드려 죄 사하기를 빌 때까지 죽지 아니하겠다는 H의 편지를 받고 나는 곧 힘 있는 답장을 썼다.',
 '젖뗀 송아지를 집으로 끌고 오던 날 어깨춤을 추면서 소 들어간다고 고함을 지르는 첨지의 언동을 보고 사람들은 방자 같다고 하면서 허리를 잡았다.',
 '손주딸의 관한 불길한 꿈을 꾼 한머니는 백여 리 떨어진 손주딸의 집에를 가보기로 하였다. ',
 '자기는 태자의 위를 동경하거나 부러워한 적이 없으며 이 나라의 충성된 신자로서 공주와 부귀영화의 일생을 보내는 것만이 최대의 희망이었다.',
 '자신의 신변을 염려하여 빠른 걸음으로 골목을 빠져나와 침착한 의식을 회복하면서 어수선한 그의 머리속에는 그의 행동에 대한 사상이 이제 풀려나왔다.']

In [ ]:
# 결과 데이터 프레임 만들고 저장하기
import pandas as pd
train_data = [train_passage, train_summary]
df = pd.DataFrame(columns = ['input_documant', 'sentence_20%'], data = train_passage)
#df.to_csv("train.csv", mode='w') # csv저장

In [10]:
df1 = pd.DataFrame(columns = ['input_documant'], data = train_passage)
df2 = pd.DataFrame(columns = ['sentence_20%'], data = train_summary)
train_data = pd.concat([df1, df2], axis=1)
train_data.head()

,input_documant,sentence_20%
0,"지어내버린 대목부터는 흥분이 버썩 줄어지었다 ──. ""선생님! 또 기침이 나고 토...",자신을 배반한 제 계집과 세상이 엎드려 죄 사하기를 빌 때까지 죽지 아니하겠다는 H...
1,이 송아지가 젖을 떼우고 집으로 끌고 오던 날은 첨지는 개선장군이 성안 에 들어올...,젖뗀 송아지를 집으로 끌고 오던 날 어깨춤을 추면서 소 들어간다고 고함을 지르는 첨...
2,어떤 여름날 밤 손주딸에게 관한 불길한 꿈을 꾼 이 한머니는 이튿날 조반 후에 생...,손주딸의 관한 불길한 꿈을 꾼 한머니는 백여 리 떨어진 손주딸의 집에를 가보기로 하...
3,이러한 가운데서 왕후는 자기의 입장을 위태롭게 여기고 겸하여 장래 자기 의 몸으로...,자기는 태자의 위를 동경하거나 부러워한 적이 없으며 이 나라의 충성된 신자로서 공주...
4,“가겠소.” “언니가 나오시면 일러드 리겠으니 그때까지는 찾아오지 않으시는 것이 ...,자신의 신변을 염려하여 빠른 걸음으로 골목을 빠져나와 침착한 의식을 회복하면서 어수...


In [11]:
# train_data.to_csv("train.csv", mode='w', )

In [13]:
# dff = pd.read_csv('/content/train.csv')
# dff.head()

,Unnamed: 0,input_documant,sentence_20%
0,0,"지어내버린 대목부터는 흥분이 버썩 줄어지었다 ──. ""선생님! 또 기침이 나고 토...",자신을 배반한 제 계집과 세상이 엎드려 죄 사하기를 빌 때까지 죽지 아니하겠다는 H...
1,1,이 송아지가 젖을 떼우고 집으로 끌고 오던 날은 첨지는 개선장군이 성안 에 들어올...,젖뗀 송아지를 집으로 끌고 오던 날 어깨춤을 추면서 소 들어간다고 고함을 지르는 첨...
2,2,어떤 여름날 밤 손주딸에게 관한 불길한 꿈을 꾼 이 한머니는 이튿날 조반 후에 생...,손주딸의 관한 불길한 꿈을 꾼 한머니는 백여 리 떨어진 손주딸의 집에를 가보기로 하...
3,3,이러한 가운데서 왕후는 자기의 입장을 위태롭게 여기고 겸하여 장래 자기 의 몸으로...,자기는 태자의 위를 동경하거나 부러워한 적이 없으며 이 나라의 충성된 신자로서 공주...
4,4,“가겠소.” “언니가 나오시면 일러드 리겠으니 그때까지는 찾아오지 않으시는 것이 ...,자신의 신변을 염려하여 빠른 걸음으로 골목을 빠져나와 침착한 의식을 회복하면서 어수...


In [20]:
import pandas as pd
train_data = pd.DataFrame(zip(train_passage, train_summary), columns=['input_documant','sentence_20%'])

In [21]:
train_data.head()

,input_documant,sentence_20%
0,"지어내버린 대목부터는 흥분이 버썩 줄어지었다 ──. ""선생님! 또 기침이 나고 토...",자신을 배반한 제 계집과 세상이 엎드려 죄 사하기를 빌 때까지 죽지 아니하겠다는 H...
1,이 송아지가 젖을 떼우고 집으로 끌고 오던 날은 첨지는 개선장군이 성안 에 들어올...,젖뗀 송아지를 집으로 끌고 오던 날 어깨춤을 추면서 소 들어간다고 고함을 지르는 첨...
2,어떤 여름날 밤 손주딸에게 관한 불길한 꿈을 꾼 이 한머니는 이튿날 조반 후에 생...,손주딸의 관한 불길한 꿈을 꾼 한머니는 백여 리 떨어진 손주딸의 집에를 가보기로 하...
3,이러한 가운데서 왕후는 자기의 입장을 위태롭게 여기고 겸하여 장래 자기 의 몸으로...,자기는 태자의 위를 동경하거나 부러워한 적이 없으며 이 나라의 충성된 신자로서 공주...
4,“가겠소.” “언니가 나오시면 일러드 리겠으니 그때까지는 찾아오지 않으시는 것이 ...,자신의 신변을 염려하여 빠른 걸음으로 골목을 빠져나와 침착한 의식을 회복하면서 어수...


In [23]:
train_data.to_csv("train.csv", mode='w')

In [26]:
dff = pd.read_csv('/content/train.csv',index_col = 0)
dff.head()

,input_documant,sentence_20%
0,"지어내버린 대목부터는 흥분이 버썩 줄어지었다 ──. ""선생님! 또 기침이 나고 토...",자신을 배반한 제 계집과 세상이 엎드려 죄 사하기를 빌 때까지 죽지 아니하겠다는 H...
1,이 송아지가 젖을 떼우고 집으로 끌고 오던 날은 첨지는 개선장군이 성안 에 들어올...,젖뗀 송아지를 집으로 끌고 오던 날 어깨춤을 추면서 소 들어간다고 고함을 지르는 첨...
2,어떤 여름날 밤 손주딸에게 관한 불길한 꿈을 꾼 이 한머니는 이튿날 조반 후에 생...,손주딸의 관한 불길한 꿈을 꾼 한머니는 백여 리 떨어진 손주딸의 집에를 가보기로 하...
3,이러한 가운데서 왕후는 자기의 입장을 위태롭게 여기고 겸하여 장래 자기 의 몸으로...,자기는 태자의 위를 동경하거나 부러워한 적이 없으며 이 나라의 충성된 신자로서 공주...
4,“가겠소.” “언니가 나오시면 일러드 리겠으니 그때까지는 찾아오지 않으시는 것이 ...,자신의 신변을 염려하여 빠른 걸음으로 골목을 빠져나와 침착한 의식을 회복하면서 어수...


In [29]:
val_summary, val_passage = search("/content/drive/MyDrive/인공지능/아이펠톤/PoC/data/val", '/VL1.zip')

In [30]:
val_data = pd.DataFrame(zip(val_passage, val_summary), columns=['input_documant','sentence_20%'])
val_data.to_csv("val.csv", mode='w')

In [31]:
dff = pd.read_csv('/content/val.csv',index_col = 0)
dff.head()

,input_documant,sentence_20%
0,러시아 황실이 제공한 임시공관에서 오전 내내 쉬며 지냈다. 오늘 아침 우리 임시공...,러시아에서 제공한 임시공관에서 쉬면서 아침에 태극기를 발코니 위에 게양했는데 모스크...
1,위에서 살펴본 방법으로 배우자 공제건수를 종합소득과 근로소득의 경우로 나누어 추정하...,기본공제액 확대 시 배우자 기본공제액 확대에 따른 공제 규모 증가분에 평균 실효세율...
2,우리나라 공공기관의 현황 분석을 시작으로 공공기관을 체계적으로 평가하기 시작하였다....,공공기관 정부 지원 예산안 평가에서는 일부 공공기관들이 여유자금을 과다하게 보유하고...
3,"그런 점에서 볼 때, 박영민씨 경우는 이례적인 케이스로 꼽힌다. 프로게이머 특성상...",박 씨는 기초 지식이 부족해 영어 문답을 통째로 외우다시피 하며 공무원 시험 준비를...
4,도로명 주소는 지번 주소에 비해 위치 파악 시 더 많은 범위를 탐색해야 하는 단점을...,도로명 주소는 지번 주소에 비해 위치 파악 시 더 많은 범위를 탐색해야 하는 점이 ...
